In [1]:
##### REQUIRES THE DATAFRAME FOLDER TO BE NAMED 'Cohorts', WHICH INCLUDES ALL PRECOMPUTED DATAFRAMES #####
import fiber
from fiber.cohort import Cohort
from fiber.condition import Patient, MRNs
from fiber.condition import Diagnosis
from fiber.condition import Measurement, Encounter, Drug
from fiber.storage import yaml as fiberyaml
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import os
from functools import reduce 

DB Password: ········


In [4]:
#load Cohorts
Case_filtered_15_540 = pq.read_table('Cohorts/Case/Case_filtered_15_540.parquet').to_pandas()
Control_filtered_15_540 = pq.read_table('Cohorts/Control/Control_filtered_15_540.parquet').to_pandas()

In [5]:
#generic function get cohort 
def df_to_cohort(df):
    mrns = list(df.index.values)
    condition = MRNs(mrns)
    return Cohort(condition)

In [6]:
#condition= FIBER Condition 
#df_mrn = pandas Data frame with Patient MRNS 
#name_df = Name o the new Dataframe
#name_feature =Feature name 
#frequency = type of occurence: EVER, COUNT,WINDOW
#Cohort_type = Case/Control
def get_has_certain_condition(condition, df_mrn, name_feature, gap_in_days,window_in_days, frequency, cohort_type):
    #get cohort
    Onset_column = "HT_Onset" if cohort_type == "Case" else "last_encounter"
    cohort = df_to_cohort(df_mrn)
    #get cohort with condition
    cohort_condition = cohort.get(condition)
    # Prefilter condition frame, so there are only the rows of interest --> rows which fulfill the gap condition
    try:
        Filtered_DF = df_mrn.merge(cohort_condition, left_index=True, right_on = "medical_record_number")
    except:
        print("DataFrame is empty. Condition does not fit to any record!")
        raise SystemExit(0)
    Filtered_DF = Filtered_DF.set_index("medical_record_number")
    # Use only necessary columns age_in_days + HT_Onset
    Filtered_DF = Filtered_DF[["age_in_days", Onset_column]]
    Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) >= gap_in_days]
    # Filtered_DF contains all gap relevant entries
    
    # get MRNs of input file 
    #cohort_indexes = df_mrn.index
    #get 0 for no diagnosis and 1 for has diagnosis
    cohort_mrn_diagnosis=[]
    if frequency=='EVER':
        #Remove all duplicate MRNs (index)
        Filtered_DF = Filtered_DF.loc[~Filtered_DF.index.duplicated(keep='first')]
        # Set all Rows to 1, because they occur in the dataframe
        Filtered_DF["has_condition_" + name_feature] = 1
        # Prepare merging to the whole cohort
        to_merge = Filtered_DF["has_condition_" + name_feature]
        to_merge = to_merge.to_frame()
        #Merge it with a left outer join
        cohort_mrn_diagnosis = df_mrn.merge(to_merge, left_index = True, right_index = True, how="left")
        # Fill all Rows, which did not appear in the condition with 0
        cohort_mrn_diagnosis["has_condition_" + name_feature].fillna(0, inplace=True)
    elif frequency=='COUNT':
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    elif frequency=='WINDOW':
        #max window size
        Filtered_DF = Filtered_DF.loc[(Filtered_DF[Onset_column] - Filtered_DF["age_in_days"]) <= (window_in_days+gap_in_days)]
        # Group by Index (MRNs)
        Count_DF = Filtered_DF.groupby([Filtered_DF.index]).count()
        # Count random column, does not matter which
        Count_DF = Count_DF[Onset_column]
        #Convert Series to Frame
        Count_DF = Count_DF.to_frame()
        # Rename column
        Count_DF.rename(columns={Onset_column:'number_of_occurences_' + name_feature},inplace=True)
        # Merge as above
        cohort_mrn_diagnosis = df_mrn.merge(Count_DF, left_index = True, right_index = True, how="left")
        cohort_mrn_diagnosis["number_of_occurences_" + name_feature].fillna(0, inplace=True)
    else:
        print("function not available")
    #convert list cohort_mrn_diagnosis to panda
    #condition_name= 'has_condition_'+name_feature
    #col_names = ['medical_record_number',condition_name]
    #df_final=pd.DataFrame(cohort_mrn_diagnosis,columns=col_names)
    #df_final.set_index('medical_record_number', inplace=True)
    #df_final=df_mrn.merge( df_final, left_index=True, right_index=True)
    #df_final.to_parquet(name_df)
    #print(df_final)
    df_final = cohort_mrn_diagnosis
    return df_final

In [7]:
condition_ihd = (Diagnosis("410%", "ICD-9")| Diagnosis("I20%", "ICD-10") | 
                Diagnosis("I21%", "ICD-10")| Diagnosis("I22%", "ICD-10") |
                Diagnosis("I23%", "ICD-10")| Diagnosis("I24%", "ICD-10") |
                Diagnosis("I25%", "ICD-10")| Diagnosis("411%", "ICD-9") |
                Diagnosis("412%", "ICD-9")| Diagnosis("413%", "ICD-9") |
                Diagnosis("414%", "ICD-9")  
                )



In [8]:
ihd_Window = get_has_certain_condition(condition_ihd , Case_filtered_15_540, "ihd", 180 ,720, "WINDOW", "Case")


Fetching data for Diagnosis (...)


In [9]:
ihd_Window.loc[ihd_Window['number_of_occurences_ihd'] != 0]

,number_of_encounters,Age_BP_condition,Age_ICD_condition,Age_MED_condition,HT_Onset,Earliest_Condition,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_ihd
medical_record_number,,,,,,,,,,,,,
1103697264,67,19522.0,19305.0,19305.0,19305.0,Age_ICD_condition,18511,794.0,1962,Unknown,,Male,2.0
1179362152,16,NaN,27168.0,27168.0,27168.0,Age_ICD_condition,26333,835.0,1935,White,,Male,1.0
1265134213,175,29101.0,28606.0,28609.0,28606.0,Age_ICD_condition,27282,1324.0,1932,White,Unknown,Female,1.0
1331507545,77,22681.0,22429.0,22429.0,22429.0,Age_ICD_condition,21724,705.0,1952,Unknown,,Male,1.0
1423747189,110,NaN,22892.0,25118.0,22892.0,Age_ICD_condition,21092,1800.0,1948,White,Unknown,Male,1.0
1573256350,183,29383.0,29341.0,NaN,29341.0,Age_ICD_condition,26310,3031.0,1935,White,Catholic,Male,2.0
167581176,194,32725.0,NaN,32727.0,32725.0,Age_BP_condition,30121,2604.0,1929,White,Christian,Female,3.0
1705858715,41,28974.0,NaN,NaN,28974.0,Age_BP_condition,27693,1281.0,1939,Other,Catholic,Male,1.0
176805496,25,NaN,25553.0,25553.0,25553.0,Age_ICD_condition,20979,4574.0,1946,Unknown,,Female,2.0


In [11]:
ihd_Window.to_parquet('ischemic_window_2yr_case.parquet')

In [ ]:
##### Controls ####

In [12]:
ihd_Window = get_has_certain_condition(condition_ihd , Control_filtered_15_540, "ihd", 180 ,720, "WINDOW", "Control")


Fetching data for Diagnosis (...)


In [13]:
ihd_Window.loc[ihd_Window['number_of_occurences_ihd'] != 0]

,last_encounter,number_of_encounters,first_encounter,total_record_time,year_of_birth,race,religion,gender,number_of_occurences_ihd
medical_record_number,,,,,,,,,
1004367489,31053,22,30258,795,1930,Unknown,,Male,1.0
1010385424,23734,17,23058,676,1953,Unknown,Unknown,Male,2.0
1016882936,21681,46,20891,790,1959,White,Lutheran,Female,5.0
102733349,24023,99,22685,1338,1951,White,,Male,12.0
1028745050,23692,26,20048,3644,1949,White,Pt Declined,Male,2.0
103541445,24714,59,18252,6462,1947,White,Jewish,Male,5.0
1040722046,25685,119,19126,6559,1947,White,Unknown,Male,2.0
1043363103,30135,48,29389,746,1935,Ba,Protestant,Male,4.0
1049404274,22755,315,17877,4878,1956,White,Jewish,Female,14.0


In [14]:
ihd_Window.to_parquet('ischemic_window_2yr_control.parquet')